# Page Structure

## Read image from PDF file

https://github.com/Belval/pdf2image/blob/master/docs/reference.md

In [ ]:
import numpy as np
from pdf2image import convert_from_path

pil_images = convert_from_path('samples/test2pdf.pdf', size=(2500, None))
images = [np.array(pil_image) for pil_image in pil_images]

In [ ]:
from matplotlib import pyplot as plt

for image in images:
    plt.figure(figsize=(3, 6))
    plt.imshow(image)
    plt.show()

In [ ]:
def load_images_from_pdf(pdf_path, size=None):
    """
    Load OpenCV images from a PDF file.
    :param pdf_path: path of the PDF file
    :param size: the preferred size in pixels as a (width, height) tuple
    :return: list of OpenCV images
    """
    pil_images = convert_from_path(pdf_path, size)
    images = [np.array(pil_image) for pil_image in pil_images]
    return images

## Margin estimation

## Row and column profiles

In [ ]:
image = images[0]
row_profile = np.mean(image, axis=1)
column_profile = np.mean(image, axis=0)

In [ ]:
plt.figure()
plt.plot(row_profile)
plt.show()

In [ ]:
plt.figure()
plt.plot(column_profile)
plt.show()